In [2]:
import numpy as np
import matplotlib.pyplot as plt

### Helper Functions

In [174]:
def R(W, alpha):
    eValues,_ = np.linalg.eig(W)
    abscissa = max(eValues)

    return np.real(W * alpha/abscissa)

def matrixHeatMap(matrix):
    plt.imshow(matrix, cmap='viridis', interpolation='nearest')
    plt.colorbar()
    plt.title('Heatmap of Matrix')
    plt.show()

### V1 Model Class Definition

In [175]:
class V1Model:
    def __init__(self):
        # Default parameters
        self.tau = 0.02 #s
        self.m = 200 # number of quantisations of angle
        self.n = 200 # number of neurons in V1
        self.B = None # matrix of weights applied to inputs to V1 dynamics, h(theta)
        self.C = None # matrix of output weights applied to outputs from V1 dynamics
        self.sigma = 1 # amplitude of noise applied to V1 dynamics output
        self.kappa = np.pi/4 # reciprocal exponential scaling factor applied to angles to give h(theta)
        self.alpha = 0.9 # scale given to abscissa of W
        self.alpha2 = 0.9 # secondary abscissa scale of W
        self.W = None # connectivity matrix (inter-neuron weights)

        # Modelling settings
        self.dt = 0.001 #s timesteps

        # Model variables
        self.theta = 0
        self.phi = np.arange(0,self.m) * 2*np.pi / self.m
        self.h = np.zeros(self.m)
        self.r = np.zeros(self.n)
        self.noisyOutput = np.zeros(self.m)
        self.thetaEst = 0
    
    def V(self, angle):
        return np.exp((np.cos(angle) - 1) / self.kappa**2)

    def hCalc(self):
        self.h = self.V(self.phi - self.theta)
    
    def rIntegrate(self):
        self.r = self.r + self.dt/self.tau * (-self.r + np.matmul(self.W, self.r) + np.matmul(self.B, self.h))

    def calcOutput(self):
        self.noisyOutput = np.matmul(self.C, self.r) + self.sigma*np.random.normal(0,1,self.m)
    
    def estimateTheta(self):
        self.thetaEst = np.arctan(sum(self.noisyOutput * np.sin(self.phi)) / sum(self.noisyOutput * np.cos(self.phi)))
    
    def calcError(self):
        self.decodeError = np.arccos(np.cos(self.thetaEst - self.theta))
        
    def stepTime(self):
        self.hCalc()
        self.rIntegrate()
        self.calcOutput()
        self.estimateTheta()
        self.calcError()


### Model Initiations

In [178]:
def setModel1():
    model1 = V1Model()
    model1.W = np.zeros((model1.m, model1.m))
    model1.B, model1.C = np.diag(np.ones(model1.m)), np.diag(np.ones(model1.m))
    return model1 # default params with n = m, W = mxm matrix of 0s

def setModel2():
    model2 = V1Model() # W = rescaled random W+W.t
    WBar2 = np.random.normal(0,1,(model2.m, model2.m))
    model2.W = R(WBar2+WBar2.T, model2.alpha)
    model2.B, model2.C = np.diag(np.ones(model2.m)), np.diag(np.ones(model2.m))
    
    return model2

def setModel3():
    model3 = V1Model() # WIJ = V(phiI - phiJ)
    WBar3 = np.zeros((model3.m, model3.m))

    for i in range(WBar3.shape[0]):
        for j in range(WBar3.shape[1]):
            WBar3[i,j] = model3.V(model3.phi[i] - model3.phi[j])

    model3.W = R(WBar3, model3.alpha)
    model3.B, model3.C = np.diag(np.ones(model3.m)), np.diag(np.ones(model3.m))
    return model3

def setModel4():
    model4 = V1Model()

    WBar3 = np.zeros((model4.m, model4.m))
    for i in range(WBar3.shape[0]):
        for j in range(WBar3.shape[1]):
            WBar3[i,j] = model4.V(model4.phi[i] - model4.phi[j])
    
    model4.n = 2*model4.m
    WBar4 = R(WBar3, model4.alpha2)
    W = np.concatenate((WBar4, -WBar4), axis=1)
    model4.W = np.concatenate((W, W), axis=0)
    model4.B = np.concatenate((np.diag(np.ones(model4.m)), np.zeros((model4.m, model4.m))), axis=0)
    model4.C = np.concatenate((np.diag(np.ones(model4.m)), np.zeros((model4.m, model4.m))), axis=1)
    return model4

In [179]:
model1 = setModel1()
model2 = setModel2()
model3 = setModel3()
model4 = setModel4()